In [15]:
import numpy as np
import copy

In [16]:
f = open("TestCase/10_10_2_4.txt")
fa = f.readlines()
tmp = fa[0].split()
N = int(tmp[0])
D = int(tmp[1])
a = int(tmp[2])
b = int(tmp[3])
day_off = np.zeros([N,D], dtype = int)
for i in range(1,len(fa)) :
    tmp = fa[i].split()
    for j in range(len(tmp) - 1) :
        m = int(tmp[j])
        day_off[i-1,m-1] = 1

In [17]:
def All_candidate (Schedule_yesterday , day_off) :
    all_caldidate = []
    for i in range(N) :
        if Schedule_yesterday[i] < 4 and day_off[i] < 1 :
            all_caldidate.append(i)
    return all_caldidate


In [18]:
def Try (k, sub_candidate_index, all_candidate, all_sub_candidate) :
    size_sub = len(sub_candidate_index)
    size_all = len(all_candidate)
    for i in range(sub_candidate_index[k-1] + 1, size_all + size_sub - k ) :
        sub_candidate_index[k] = i
        if k == size_sub - 1 :
            sub_candidate = []
            for j in sub_candidate_index[1:size_sub] :
                sub_candidate.append(all_candidate[j-1])
            all_sub_candidate.append(sub_candidate)
        else :
            Try (k+1, sub_candidate_index, all_candidate, all_sub_candidate)
        

In [19]:
def generate_candidate (all_candidate) :
    all_combination_4a = [] # Tap tat ca to hop 4*a cua all_candidate
    b = [0] * (4*a + 1)
    Try (1, b, all_candidate, all_combination_4a)
    all_combination = []
    for each_combination_4a in all_combination_4a :
        all_combination_a = []
        c = [0] * (a+1)
        Try (1, c, each_combination_4a, all_combination_a)
        for each_combination_a in all_combination_a :
            tmp = []
            tmp += each_combination_a
            tmp += [j for j in each_combination_4a if j not in each_combination_a]
            all_combination.append(tmp)
    return all_combination


In [20]:
Schedule = np.zeros([N,D+1], dtype=int)
num_night = [0]*N
min_num_night = 1000000
num_night_to_now = [0] * N

In [21]:
def update() :
    global min_num_night
    for i in range(N) :
        num_night[i] = 0
        for j in range(1,D+1) :
            if Schedule[i,j] == 4 :
                num_night[i] += 1
    if min_num_night > max(num_night) :
        min_num_night = max(num_night)
        print(Schedule)
        print(min_num_night)

In [22]:
def check (k) :
    global min_num_night
    num_night_tmp = [0] *N
    for i in range(N) :
        for j in range(1,k+1) :
            if Schedule[i,j] == 4 :
                num_night_tmp[i] += 1
    if max(num_night_tmp) >= min_num_night :
        return False
    else :
        return True

In [23]:
def up_to_now(k) :
    global num_night_to_now
    for i in range(N) :
        num_night_to_now[i] = 0
        for j in range(1,k+1) :
            if Schedule[i,j] == 4 :
                num_night_to_now[i] += 1

In [24]:
def heuristic (candidate) :
    global num_night_to_now
    num_night_to_now_tmp = copy.deepcopy(num_night_to_now)
    for i in candidate[0:a] :
        num_night_to_now_tmp[i] += 1
    # return max(num_night_to_now_tmp) - min(num_night_to_now_tmp)
    num_night_to_now_tmp = np.array(num_night_to_now_tmp)
    return np.std(num_night_to_now_tmp)

In [25]:
stop = False
lower_bound = 0
if (a*D) % N == 0 :
    lower_bound = int(a*D/ N)
else :
    lower_bound = int((a*D)/N) + 1

In [26]:
def Try_main (k) :
    global num_night_to_now
    global stop
    global lower_bound
    if stop == True :
        return
    for i in range(N) :
        Schedule[i,k] = 0
    all_candidate = All_candidate(list(Schedule[:,k-1]), list(day_off[:,k-1])) 
    if len(all_candidate) < 4*a :
        return 
    up_to_now(k-1)
    all_combination = generate_candidate (all_candidate)
    all_combination.sort(key=heuristic, reverse=False)
    for combination in all_combination :
        for i in range(a) :
            Schedule[combination[i], k] = 4
        for i in range(a,4*a) :
            Schedule[combination[i], k] = 1
        if k == D :
            update()
            if min_num_night == lower_bound :
                stop = True
                return
        else :
            if check(k) == True :
                Try_main(k+1)

In [27]:
print(lower_bound)

2


In [28]:
Try_main(1)
print(min_num_night)

[[0 4 0 1 1 1 1 4 0 4 0]
 [0 1 4 0 1 1 1 1 4 0 1]
 [0 1 1 4 0 1 4 0 1 1 1]
 [0 1 1 1 4 0 1 1 4 0 1]
 [0 1 1 1 1 4 0 4 0 4 0]
 [0 1 4 0 4 0 1 1 1 1 1]
 [0 4 0 1 1 1 1 1 1 1 4]
 [0 0 1 1 1 4 0 1 1 1 4]
 [0 1 1 1 1 1 4 0 1 1 1]
 [0 0 1 4 0 1 1 1 1 1 1]]
3


KeyboardInterrupt: 